# Testing de módulo `gee_objects.py`

### Imports

In [12]:
import geopandas as gpd
import ee
import os
import gee_objects as gee

# ee.Authenticate()

ee.Initialize()

### CRS

In [2]:
crs_metre = '32719'
crs_degree = '4326'

### Data retrieval

In [3]:
comunas_path = os.path.join('Data', 'Comunas')
comunas = gpd.read_file(comunas_path).to_crs(crs_degree)

regiones_path = os.path.join('Data', 'Regional')
regiones = gpd.read_file(regiones_path).to_crs(crs_degree)


### Data selection

In [4]:
comuna_name = 'Petorca'

comuna = {
    'name': comuna_name,
    'geometry': comunas[comunas['Comuna'] == comuna_name].geometry
}

## Testing

In [5]:
geo = gee.GEERegion(comuna['geometry'], 
                    name=comuna['name'], 
                    crs=crs_degree)

dates = ('2021-01-01', '2021-12-31')
day_freq = 30

images = geo.get_images(*dates, dayfreq=day_freq, satt='Landsat-8')

indexes = ['NDVI', 'NDWI', 'NMDI', 'NDDI']
indexes = {index: geo.get_indexes(index, images) for index in indexes}


In [6]:
img_months = {i: [images[i]] + [indexes[index][i] for index in indexes]
              for i in range(len(images))}

geo.visualize(img_months[0], clip=True)

Map(center=[-32.19044545853853, -70.86993455011672], controls=(WidgetControl(options=['position', 'transparent…

In [7]:
%load_ext autoreload
%autoreload 2

In [13]:
import segment

snic_obj = segment.Snic(30, 1, 4, crs_degree)
clusters = snic_obj.run(img_months[0][0], returnbands=False)

In [14]:
mean_bands = snic_obj.run(img_months[0][0], returnbands=True)


In [15]:
geo.visualize(img_months[0]+[clusters]+[mean_bands], clip=True)

Map(center=[-32.19044545853853, -70.86993455011672], controls=(WidgetControl(options=['position', 'transparent…